# Decoding analysis example notebook

In [25]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from core import *
import time
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

dir = r"D:\clickbait-mmz"
save_dir = r"C:\Users\smearlab\analysis_code\EncodingModels\outputs"

## Setting up an example session

In [26]:
# setting up parameters
mouse = '6002'
session = '6'
window_size = 2.0  # seconds
step_size = 2  # seconds
fs = 30000  # sampling frequency for spike data
sfs = 1000  # sampling frequency for sniff data
use_units = 'good'  # can be 'all', 'good', or a specific unit number
k_CV = 3  # number of cross-validation folds
n_blocks = 3
shift = 2 # shift for construction of null distribution (0 means no shift)
model_input = 'neural'
use_behaviors = ['position_x', 'position_y', 'velocity_x', 'velocity_y', 'sns']  # behaviors to use in the model

save_dir = os.path.join(save_dir, mouse, session, f'shift_{shift}')
os.makedirs(save_dir, exist_ok=True)

**Data preprocessing**

In [27]:
# Preprocessing the data to get spike counts, behavioral variables and names, trial IDs, neuron names, and neuron info from manual curation
counts, variables, variable_names, trial_ids, neu_names, neu_info = preprocess(dir, save_dir, mouse, session, window_size, step_size, use_units)

***choose which region to analyze from 'OB' or 'HC'***

This code also performs the circular shift of the spike rates if shift > 0 for the construction of the null distribution for hypothesis testing

In [28]:
# get the neurons for the correct area
use_units = [key for key, value in neu_info.items() if value['area'] == 'HC']
spike_rates = counts[:, np.isin(neu_names, use_units)]
if shift > 0:
    random_roll = np.random.randint(0, spike_rates.shape[0])
    print(f'rolling spike rates by {random_roll} time bins')
    spike_rates = np.roll(spike_rates, random_roll, axis=0)
print(f'spike rates shape (# time bins, # neurons): {spike_rates.shape}')

rolling spike rates by 761 time bins
spike rates shape (# time bins, # neurons): (917, 29)


***Extracting the behavioral variables***


In [29]:
print(f'variables: {variable_names}')

behavior_components = []
if 'position_x' in use_behaviors:
    pos_x = np.array(variables[variable_names.index('position_x')])
    behavior_components.append(pos_x)

if 'position_y' in use_behaviors:
    pos_y = np.array(variables[variable_names.index('position_y')])
    behavior_components.append(pos_y)

if 'velocity_x' in use_behaviors:
    vel_x = np.array(variables[variable_names.index('velocity_x')])
    behavior_components.append(vel_x)

if 'velocity_y' in use_behaviors:
    vel_y = np.array(variables[variable_names.index('velocity_y')])
    behavior_components.append(vel_y)

if 'sns' in use_behaviors:
    sniff_rate = np.array(variables[variable_names.index('sns')])
    behavior_components.append(sniff_rate)

behavior = np.stack(behavior_components, axis=1)
print(f'behavior shape (# time bins, # dims): {behavior.shape}')

variables: ['position_x', 'position_y', 'velocity_x', 'velocity_y', 'sns', 'latency', 'phase', 'speed', 'click']
behavior shape (# time bins, # dims): (917, 5)


***Building list of arguements to pass to the model***

This is where the input $\bold{X}$ and output $y$ variables for the model are defined.

$y = f(\bold{X})$ where $f(\cdot)$ is learned from the data to minimize some cost function.



In [30]:
# looping through the folds of cross-validation to build datasets
arg_list = []

for k in range(k_CV):
    rates_train, rates_test, train_switch_ind, test_switch_ind = cv_split(spike_rates, k, k_CV, n_blocks)
    behavior_train, behavior_test, _, _ = cv_split(behavior, k, k_CV, n_blocks)

    current_save_path = os.path.join(save_dir, "model fits")
    os.makedirs(current_save_path, exist_ok=True)


    # Decide which data to use for input (X) and output (y)
    if model_input == 'neural':
        y_train = behavior_train
        y_test = behavior_test

        X_train = rates_train
        X_test = rates_test
    elif model_input == 'behavioral':
        y_train = rates_train
        y_test = rates_test

        X_train = behavior_train
        X_test = behavior_test

    arg_list.append((X_train, X_test, y_train, y_test, train_switch_ind, test_switch_ind, current_save_path, None, shift))


In [31]:
def plot_preds(targets, predictions, test_switch_ind, sequence_length, save_path, k, shift):
    adjusted_test_switch_ind = [ind - sequence_length * k for k, ind in enumerate(test_switch_ind)]
    behavior_dim = targets.shape[1]
    _, ax = plt.subplots(behavior_dim, 1, figsize=(20, 10), sharex= True)
    if behavior_dim == 1:
        ax = [ax]
    for i in range(behavior_dim):
        ax[i].plot(targets[:, i], label='True', color = 'crimson')
        ax[i].plot(predictions[:, i], label='Predicted')
        for ind in adjusted_test_switch_ind:
            ax[i].axvline(ind, color='grey', linestyle = '--', alpha=0.5)

    if behavior_dim > 4:
        # remove the y-axis ticks 
        for a in ax:
            a.set_yticks([])
    plt.xlabel('Time')
    ax[0].legend(loc = 'upper right')

    sns.despine()
    plt.savefig(os.path.join(save_path, f'lstm_predictions_k_{k}_shift_{shift}.png'), dpi=300)
    plt.close()

In [32]:
def process_fold(X_train, X_test, y_train, y_test, train_switch_ind, test_switch_ind, current_save_path,
        device = None, shift = 0, hidden_dim = 8, num_layers = 2, dropout = 0.1, sequence_length = 3, target_index = -1, 
        batch_size = 64, lr = 0.01, num_epochs = 300, patience = 10, min_delta = 0.01, factor = 0.5, plot_predictions = True):
    
    # Set the device
    if device:
        os.environ['CUDA_VISIBLE_DEVICES'] = device
        device = torch.device('cuda:0')
    else:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")



    # Create the model
    lstm_model = LSTMDecoder(input_dim= X_train.shape[1], hidden_dim=hidden_dim, output_dim=y_train.shape[1], num_layers=num_layers, dropout = dropout).to(device)


    # Prepare the training data for LSTM
    blocks = [(train_switch_ind[i], train_switch_ind[i + 1]) for i in range(len(train_switch_ind) - 1)]
    train_dataset = SequenceDataset(X_train, y_train, blocks, sequence_length, target_index)
    train_loader = DataLoader(train_dataset, batch_size=min(batch_size, len(train_dataset)), shuffle=False, num_workers=0, pin_memory=True, prefetch_factor=None)


    # Training the LSTM model
    start_train = time.time()
    trained_lstm_model, lstm_history = train_LSTM(lstm_model, train_loader, device, lr=lr, epochs=num_epochs, patience=patience, min_delta=min_delta, factor=factor, verbose=False)
    print(f"\nTraining time: {time.time() - start_train:.2f}s fold={k} shift={shift}", flush=True)
    # Free up memory
    del lstm_model, train_dataset, train_loader


    # Plot loss
    if plot_predictions:
        plot_training(lstm_history, current_save_path, shift, k)


    # Prepare the test data for LSTM
    test_blocks = [(test_switch_ind[i], test_switch_ind[i + 1]) for i in range(len(test_switch_ind) - 1)]
    test_dataset = SequenceDataset(X_test, y_test, test_blocks, sequence_length, target_index)
    test_loader = DataLoader(test_dataset, batch_size=min(batch_size, len(test_dataset)), num_workers=0, pin_memory=True)


    # Predict on the test set
    trained_lstm_model.eval()
    predictions = []
    targets = []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            preds = trained_lstm_model(X_batch)
            predictions.append(preds.cpu().numpy())
            targets.append(y_batch.cpu().numpy())
    predictions = np.concatenate(predictions, axis=0)
    targets = np.concatenate(targets, axis=0)

    # Clean up
    del trained_lstm_model, test_dataset, test_loader

    plot_preds(targets, predictions, test_switch_ind, sequence_length, current_save_path, k, shift)

    diffs = predictions - targets
    rmse = np.sqrt(np.mean(diffs ** 2, axis=0))


    torch.cuda.empty_cache()
    print(f"Total time {time.time() - start_train:.2f}s for fold={k} shift={shift}\n\n\n", flush=True)


    return rmse, targets, predictions

***Running the model across all the folds and saving the results***

In [33]:
results = []
results_save_dir = os.path.join(save_dir, "results")
os.makedirs(results_save_dir, exist_ok=True)
for k in range(k_CV):
    print(f"Processing fold {k + 1}/{k_CV}...")
    rmse, targets, predictions = process_fold(*arg_list[k])

    # Define file paths for saving results
    results_file = f'results_shift{shift}_fold{k}.npz'
    results_path = os.path.join(results_save_dir, results_file)
    
    # Save rmse as compressed .npz file
    np.savez_compressed(results_path, rmse=rmse, targets=targets, predictions=predictions)


    # Store just the path in the results table
    results.append({
        'mouse': mouse,
        'session': session,
        'shift': shift,
        'fold': k,
        'results_file': results_file,
    })

# convert to DataFrame and save as CSV
df = pd.DataFrame(results)
df.to_csv(os.path.join(results_save_dir, 'results.csv'), index=False)




Processing fold 1/3...
Using device: cuda

Training time: 2.46s fold=0 shift=2
Total time 3.69s for fold=0 shift=2



Processing fold 2/3...
Using device: cuda

Training time: 3.38s fold=1 shift=2
Total time 5.19s for fold=1 shift=2



Processing fold 3/3...
Using device: cuda

Training time: 2.99s fold=2 shift=2
Total time 4.11s for fold=2 shift=2





# Statistical analysis to compare the models

rmse is a list with n_folds elements, each element is a list with rmse values for each fold. The index of the RMSE value corresponds to the target dimension.

In [45]:
from scipy.stats import ranksums

n_dims = 5

session_dir = r"C:\Users\smearlab\analysis_code\EncodingModels\outputs\6002\6"

shifts = os.listdir(session_dir)
n_shifts = len(shifts)

print(f'Number of shifts: {n_shifts - 1}')
for dim in range(n_dims):
    true_rmse = []
    null_rmse = []
    for shift in range(n_shifts):
        shift_dir = os.path.join(session_dir, f'shift_{shift}', 'results')
        n_folds = len([f for f in os.listdir(shift_dir) if f.startswith('results') and f.endswith('.npz')])
        for fold in range(n_folds):
            rmse = np.load(os.path.join(shift_dir, f'results_shift{shift}_fold{fold}.npz'))['rmse'][dim]

            if shift == 0:
                true_rmse.append(rmse)
            else:
                null_rmse.append(rmse)

    stat, p = ranksums(true_rmse, null_rmse)
    print(f'dim {dim}: true - null RMSE = {np.mean(true_rmse) - np.mean(null_rmse):.4f}, p-value = {p:.4f}, statistic = {stat:.4f}')













Number of shifts: 2
dim 0: true - null RMSE = -14.6328, p-value = 0.0201, statistic = -2.3238
dim 1: true - null RMSE = -6.1701, p-value = 0.1967, statistic = -1.2910
dim 2: true - null RMSE = -0.0024, p-value = 0.7963, statistic = -0.2582
dim 3: true - null RMSE = 0.0127, p-value = 0.4386, statistic = 0.7746
dim 4: true - null RMSE = 0.5347, p-value = 0.1213, statistic = 1.5492
